In [1]:
from dataloader import SignalDataset,dataloader 
from param import dataset_path,sample_universe_size,sampling_rate
from utils import prepare_data, mix_signals,load_audio,load_music
import torch
from torch.utils.data import ConcatDataset
import os
from param import (
    dataset_path,
    sample_universe_size,
    data_dir,
    hyper_parameters as hp,
    model_1,
)
from sklearn.metrics import f1_score, precision_score, recall_score

In [2]:
#Call this code in train and evaluate functions (to be deleted later)
data_dir = r"../data"
model_dir = r"../models"
data_file = os.path.join(data_dir, "dataset.pth")
if "dataset.pth" in os.listdir(data_dir):
    combined_dataset = torch.load(data_file)
else:
    # Prepare data
    combination_paths = prepare_data(dataset_path)
    sampled_df = combination_paths.sample(
        frac=sample_universe_size, random_state=42, ignore_index=True
    )
    datasets_music = SignalDataset(sampled_df, data_type="music")
    datasets_speech = SignalDataset(sampled_df, data_type="speech")
    datasets_mixture = SignalDataset(sampled_df, data_type="mixture")
    combined_dataset = ConcatDataset(
        [datasets_music, datasets_speech, datasets_mixture]
    )
    torch.save(combined_dataset, data_file)


In [3]:
train_loader, test_loader = dataloader(
    datasets=combined_dataset,
    train_ratio=hp["train_ratio"],
    train_batch_size=hp["batch_size"],
    test_batch_size=1,
)

In [4]:
def calculate_metrics(predictions, targets):
    # Convert predictions and targets to tensors if they're not already
    if not isinstance(predictions, torch.Tensor):
        predictions = torch.tensor(predictions)
    if not isinstance(targets, torch.Tensor):
        targets = torch.tensor(targets)

    # Calculate true positives, false positives, and false negatives
    true_positives = torch.sum(predictions * targets).float()
    false_positives = torch.sum(predictions * (1 - targets)).float()
    false_negatives = torch.sum((1 - predictions) * targets).float()

    # Calculate precision
    precision = true_positives / (true_positives + false_positives + 1e-10)

    # Calculate recall
    recall = true_positives / (true_positives + false_negatives + 1e-10)

    # Calculate F1 score
    f1_score = 2 * (precision * recall) / (precision + recall + 1e-10)

    return precision.item(), recall.item(), f1_score.item()



In [5]:
# Assuming your model architecture is defined in a separate file or module
from MTL_w_cascade_info import MtlCascadeModel
from param import hyper_parameters as hp
# Path to the state dict file
state_dict_path = '/home/hparashar/mtech_projects/speech_understanding/Speech-Understanding-Minor/model/model_0.001.pth'
hyper_parameters = {
    "n_layers": 1,
    "sp_hidden_nodes": 20,
    "n_sp_hidden_lyrs": 1,
    "mu_hidden_nodes": 20,
    "n_mu_hidden_lyrs": 1,
    "smr_hidden_nodes": 20,
    "n_smr_hidden_lyrs": 1,
    "n_epochs": 100,
    "batch_size": 20,
    "train_ratio": 0.8,
}
# Create an instance of your model
model = MtlCascadeModel(hyper_parameters)

# Load the state dict onto the model
state_dict = torch.load(state_dict_path)
model.load_state_dict(state_dict)

# Now, you can use your model for inference or further training


<All keys matched successfully>

In [12]:
def evaluate(model, test_loader):
    model.eval()
    c = 0
    for data in test_loader:
        # Unpack the data
        feature, targets = data
        # Forward pass
        predictions = model(feature)
        c = c+1
        if c == 2:
            break
    return predictions, targets

In [13]:
pred, targ =evaluate(model, test_loader)

In [14]:
pred_num = pred[0].detach().numpy()
print(pred[0].detach().numpy())
print(pred[1].detach().numpy().astype(float))
print(pred[2].detach().numpy())


[[7.656785e-05]]
[[5.27256353e-13]]
[[0.34927204 0.34968373]]


In [9]:
pred_num = pred[0].detach().numpy()
print(pred_num)

[[1.]]


In [10]:
targ

('speech',)

In [11]:
# Example usage:
predictions = torch.tensor([0, 1, 1, 0, 1])
targets = torch.tensor([0, 1, 0, 1, 1])

precision, recall, f1_score = calculate_metrics(predictions, targets)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1_score)


Precision: 0.6666666865348816
Recall: 0.6666666865348816
F1 Score: 0.6666666865348816
